In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from processing import Process

conf = SparkConf()
# conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

procdata = Process()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/29 12:22:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/29 12:22:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
geralprouni = pd.DataFrame(pd.read_xml("pure_data/geralprouni.xml"))

quantidade_bolsas = geralprouni[['bolsa_integral_cotas', 'bolsa_integral_ampla', 'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)
nota_bolsa = geralprouni[['nota_integral_ampla', 'nota_integral_cotas', 'nota_parcial_ampla', 'nota_parcial_cotas']].mean(axis=1)

prouni = pd.DataFrame({
    'mensalidade': geralprouni['mensalidade'], 
    'estado': geralprouni['uf_busca'], 
    'municipio': geralprouni['cidade_busca'].map(procdata.no_accents_lower),
    'curso': geralprouni['nome'], 
    'quantidade_bolsas': quantidade_bolsas,
    'nota_bolsa': nota_bolsa
})
#geralprouni.head(5)
#print(geralprouni['cidade_busca'].head())
print(prouni.__len__(), "rows")
prouni.sample(n=10)

41447 rows


,mensalidade,estado,municipio,curso,quantidade_bolsas,nota_bolsa
4554,1195.74,PR,curitiba,Marketing,6.0,508.14
28655,295.00,SP,sao paulo,Gestão Ambiental,1.0,555.80
2163,1474.40,PR,curitiba,Engenharia Elétrica,5.0,568.52
23466,327.14,PE,garanhuns,Matemática,1.0,595.16
578,2200.00,MG,belo horizonte,Engenharia de Produção,11.0,450.00
21873,358.20,SP,sao paulo,Geografia,2.0,587.31
30828,279.00,SP,sao paulo,Gestão Financeira,1.0,489.76
38509,237.00,PR,realeza,Geografia,1.0,516.84
18612,459.60,PE,recife,Ciência da Computação,7.0,602.11
10121,838.00,MT,cuiaba,Enfermagem,3.0,577.92


In [3]:
addprouni = pd.read_json("pure_data/prouni.json")

adicionalprouni = pd.DataFrame({
    'curso': addprouni['NOME_CURSO_BOLSA'],
    'estado': addprouni['SIGLA_UF_BENEFICIARIO_BOLSA'],
    'municipio': addprouni['MUNICIPIO_BENEFICIARIO_BOLSA'].map(procdata.no_accents_lower),
    'sexo': addprouni['SEXO_BENEFICIARIO_BOLSA'].map(lambda sexo: 'Masculino' if sexo == 'M' else 'Feminino'),
    'raca': addprouni['RACA_BENEFICIARIO_BOLSA'].map(lambda raca: 'Outro' if raca not in ['Parda', 'Branca', 'Preta'] else raca),
    'deficiente': addprouni['BENEFICIARIO_DEFICIENTE_FISICO'].map(lambda deficiente: 'Sim' if deficiente == 'S' else 'Não'),
    'idade': addprouni['DT_NASCIMENTO_BENEFICIARIO'].map(procdata.get_age_from_birthdate)
})
#addprouni.head(3)
print(addprouni.columns)
print(adicionalprouni.__len__(), "rows")
adicionalprouni.sample(n=10)

Index(['ANO_CONCESSAO_BOLSA', 'CODIGO_EMEC_IES_BOLSA', 'NOME_IES_BOLSA',
       'TIPO_BOLSA', 'MODALIDADE_ENSINO_BOLSA', 'NOME_CURSO_BOLSA',
       'NOME_TURNO_CURSO_BOLSA', 'CPF_BENEFICIARIO_BOLSA',
       'SEXO_BENEFICIARIO_BOLSA', 'RACA_BENEFICIARIO_BOLSA',
       'DT_NASCIMENTO_BENEFICIARIO', 'BENEFICIARIO_DEFICIENTE_FISICO',
       'REGIAO_BENEFICIARIO_BOLSA', 'SIGLA_UF_BENEFICIARIO_BOLSA',
       'MUNICIPIO_BENEFICIARIO_BOLSA'],
      dtype='object')
236636 rows


,curso,estado,municipio,sexo,raca,deficiente,idade
99124,Psicologia,GO,goiania,Feminino,Parda,Não,23
109736,Gestão De Recursos Humanos,PE,passira,Feminino,Parda,Não,27
212639,Engenharia,MG,uberlandia,Masculino,Preta,Não,22
64783,Engenharia Civil,SC,palhoca,Masculino,Branca,Não,24
9009,Educação Física,SC,xanxere,Masculino,Branca,Não,22
66086,Psicologia,BA,feira de santana,Feminino,Parda,Não,32
9451,Computação,PR,almirante tamandare,Masculino,Preta,Não,50
8769,Fisioterapia,SC,belmonte,Feminino,Parda,Não,28
108923,Educaçao Física,PR,curitiba,Masculino,Parda,Não,22
49640,Pedagogia,MG,passos,Feminino,Parda,Não,44


In [4]:
edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")

educacaobasica = pd.DataFrame({
    'estado': edbasica['SG_UF'],
    'municipio': edbasica['NO_MUNICIPIO'].map(procdata.no_accents_lower),
    'esgoto_inexistente': edbasica['IN_ESGOTO_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'energia_inexistente': edbasica['IN_ENERGIA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'agua_inexistente': edbasica['IN_AGUA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'acesso_internet': edbasica['IN_INTERNET'].map(procdata.get_label_from_floatbool),
    'faz_exame_selecao': edbasica['IN_EXAME_SELECAO'].map(procdata.get_label_from_floatbool),
    'especializada_deficientes': edbasica['IN_ESP'].map(procdata.get_label_from_floatbool),
    'ensino_tecnico': edbasica['IN_PROF_TEC'].map(procdata.get_label_from_floatbool)
})
#dfedbasica = spark.createDataFrame(geralprouni)
#edbasica.head(10)
print(edbasica[edbasica['SG_UF']=='MG'].sample(10))
print(educacaobasica.__len__(), "rows")
educacaobasica.sample(10)

22/05/29 12:23:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(scavenge), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
22/05/29 12:23:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(global, scavenge), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
/tmp/ipykernel_7147/4192748270.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")


        NU_ANO_CENSO NO_REGIAO  CO_REGIAO         NO_UF SG_UF  CO_UF  \
114268          2021   Sudeste          3  Minas Gerais    MG     31   
123963          2021   Sudeste          3  Minas Gerais    MG     31   
111577          2021   Sudeste          3  Minas Gerais    MG     31   
110570          2021   Sudeste          3  Minas Gerais    MG     31   
122128          2021   Sudeste          3  Minas Gerais    MG     31   
123982          2021   Sudeste          3  Minas Gerais    MG     31   
115945          2021   Sudeste          3  Minas Gerais    MG     31   
105309          2021   Sudeste          3  Minas Gerais    MG     31   
119148          2021   Sudeste          3  Minas Gerais    MG     31   
116148          2021   Sudeste          3  Minas Gerais    MG     31   

           NO_MUNICIPIO  CO_MUNICIPIO                   NO_MESORREGIAO  \
114268   Frei Lagonegro       3126950                 Vale do Rio Doce   
123963        Ritápolis       3156106              Campo da

,estado,municipio,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
150372,SP,campinas,Não,Não,Não,Sim,Não,Sim,Não
220455,DF,brasilia,Não,Não,Não,Sim,Não,Sim,Não
9273,AM,uarini,Sim,Sim,Não,Não,Não,Não,Não
143867,RJ,rio de janeiro,Não,Não,Não,Sim,Sim,Não,Não
62139,RN,sao jose do campestre,Não,Não,Não,Não,Não,Não,Não
143547,RJ,rio de janeiro,Não,Não,Não,Não,Não,Não,Não
150907,SP,candido rodrigues,Não,Não,Não,Sim,Não,Sim,Não
215449,GO,bonopolis,Não,Não,Não,Sim,Não,Sim,Não
205035,RS,porto alegre,Não,Não,Não,Sim,Não,Sim,Não
71366,PE,escada,Não,Não,Não,Sim,Não,Não,Não


In [5]:
# SPARK

# df = spark.createDataFrame(prouni)
# dfadicionalprouni = spark.createDataFrame(adicionalprouni.drop_duplicates(subset=['estado', 'curso']))
# dfeducacaobasica = spark.createDataFrame(educacaobasica.drop_duplicates(subset=['estado']))
# df_joinadicional = df.join(dfadicionalprouni, on=['estado', 'curso'], how='left')
# df_joinedbasica = df_joinadicional.join(dfeducacaobasica, on='estado', how='left')
# dfmaster = df_joinedbasica.dropna()

# PANDAS

df = prouni.copy()
dfadicionalprouni = adicionalprouni.drop_duplicates(subset=['estado', 'curso', 'municipio'])
dfeducacaobasica = educacaobasica.drop_duplicates(subset=['estado', 'municipio'])
df_joinadicional = df.merge(dfadicionalprouni, on=['estado', 'curso', 'municipio'], how='left')
df_joinedbasica = df_joinadicional.merge(dfeducacaobasica, on=['estado', 'municipio'], how='left')
dfmaster = df_joinedbasica.dropna()

In [6]:
# SPARK
# dfmaster.coalesce(1).write.csv('processed_data/prouni.csv')

# PANDAS
dfmaster.to_csv('processed_data/prouni-processado.csv')